In [13]:

from sklearn.linear_model import LogisticRegression
import numpy as np
import glob

In [14]:
# Create the Random Forest Regressor model
model = LogisticRegression( max_iter=3000)
need_maintenance = 30

In [15]:
from sklearn.metrics import accuracy_score, classification_report

# Aggregate all my data
x_train, y_train = [], []
for train_path in glob.glob('..\data\\train_FD00[0-9].txt'):
    # Get aggregate path for the same train file
    agg_path = train_path.replace('.txt', '_agg.txt')
    
    # Get both trainiing and aggregate data
    train_data = np.genfromtxt(train_path, delimiter=' ')
    agg_data = np.genfromtxt(agg_path, delimiter=' ')

    # Merge the 2
    train_data = np.hstack((train_data, agg_data))
    
    unique = []
    curr_id = -1
    last = []
    
    # Find the last item for each id
    for d in train_data:
        if d[0] != curr_id:
            if curr_id != -1:
                unique.append(list(last))
            curr_id = d[0]
        last = d
    unique.append(list(last))
    
        
    # Set labels as last cycle - curr cycle (Find a better way)
    labels = np.array([(unique[(int(item[0]) - 1) % len(unique)][1] - item[1]) < need_maintenance for item in train_data])
    
    y_train.extend(labels)
    x_train.extend(train_data[:, 1:])
    

In [16]:
# Fit model
model.fit(x_train, y_train)

# Make predictions on the training data
y_pred = model.predict(x_train)

c:\Users\chris\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
# Evaluate the model on training data
accuracy = accuracy_score(y_train, y_pred)
report = classification_report(y_train, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)

Accuracy: 0.94
Classification Report:
              precision    recall  f1-score   support

       False       0.96      0.97      0.96    139089
        True       0.79      0.71      0.75     21270

    accuracy                           0.94    160359
   macro avg       0.87      0.84      0.85    160359
weighted avg       0.93      0.94      0.93    160359



In [18]:
# Aggregate all my data
x_test, y_test = [], []
for test_path in glob.glob('..\data\\test_FD00[0-9].txt'):
    agg_path = test_path.replace('.txt', '_agg.txt')
    rul_path = test_path.replace('test', 'RUL')
    
    test_data = np.genfromtxt(test_path, delimiter=' ')
    agg_data = np.genfromtxt(agg_path, delimiter=' ')
    
    labels = np.array([x < need_maintenance for x in np.genfromtxt(rul_path)])
    y_test.extend(labels)

    test_data = np.hstack((test_data, agg_data))
    
    # Get item with last cycle for each id
    unique = []
    curr_id = -1
    last = []
    
    for d in test_data:
        if d[0] != curr_id:
            if curr_id != -1:
                unique.append(list(last))
            curr_id = d[0]
        last = d        
    unique.append(list(last))
    
    unique = [row[1:] for row in unique]
    
    x_test.extend(unique)
    

In [19]:
# Predict on my data
y_pred = model.predict(x_test)

In [20]:
# Evaluate the model on test data
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)

Accuracy: 0.79
Classification Report:
              precision    recall  f1-score   support

       False       0.79      1.00      0.88       550
        True       1.00      0.04      0.09       157

    accuracy                           0.79       707
   macro avg       0.89      0.52      0.48       707
weighted avg       0.83      0.79      0.70       707

